In [ ]:
#Experiment 2-Task 1: Tag all the parts of speech for the given sentences using nltk library or library of your choice.
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


In [ ]:

from nltk.corpus import brown
from gensim.models import Word2Vec
import multiprocessing



In [ ]:
sentences=brown.sents()
print(sentences[:3])

In [ ]:
EMB_DIM = 300

w2v = Word2Vec(sentences, size=EMB_DIM, window=5, min_count=5, negative=15,
                iter=10, workers=multiprocessing.cpu_count())

In [ ]:
word_vectors=w2v.wv #get trained embeddings - an KeyedVectors instance
result=word_vectors.similar_by_word("Knowledge")
print("Most similar to 'Knowledge', result[:3])


result=word_vectors.similar_by_word("Above")
print("Most similar to 'Above', result[:3])


result=word_vectors.similar_by_word("sakuni")
print("Most similar to 'sakuni', result[:3])

In [ ]:
#Experiment 1-Task 2: Train a word2vec skip gram model with gensim package and
# compare the performance with cbow model based on known similarity between words.
from nltk.corpus import conl1200
from gensim.models import Word2Vec
import tensorflow as tf
from tensorflow.keras.layers import Dense, Embedding, Activation, Flatten
from tensorflow.keras import Sequential
from tensorflow.Keras.utils import to_categorical
import numpy as np
import collections


# getting tokenised and part-of-speech tagged data from the corpus
train_words = conl12000.tagged_words("train.txt")
test_words = conl12000.tagged_words("test.txt")
print(train_words[:10])

In [ ]:
def get_tag_vocabulary(tagged_words)
"""
Accepts text in the form of (word,pos) tuples and returns a dictionary
mapping POS-tags to unique ids.
"""
tag2id = {}
for item in tagged_words:
tag = item[1]
tag2id.setdefault(tag, len(tag2id))
return tag2id

#the word_vectors.vocab dictionary stores Vocab objects, rather than integers
# but we would like our dictionary to map words to ints
word2id = {k:v.index for k, v in word_vectors.vocab.items()}
tag2id = get_tag_vocabulary(train_words)


In [ ]:

def get_int_data(tagged_words, word2id, tag2id):

X,Y =[], []
unk_count =0

for word,tag in tagged_words:
Y.append(tag2id.get(tag))
if word in word2id:
X.append(word2id.get(word))
else:
unk_count += 1
print("Data created. Percentage of unknown words: %.3f" % (unk_count/len(tagged_words)))
return np.array(X), np.array(Y)

X_train, Y_train = get_int_data(train_words,word2id, tag2id)
X_trest, Y_test = get_int_data(test_words,word2id, tag2id)


# we need to one-hot encode the tag indexes
Y_train, Y_test =to_categorical(Y_train), to_categorical(Y_test)

In [ ]:

def add_new_word(new_word, new_vector, new_index, embedding_matrix,word2id):
"""
Adds a new word to the existing matrix of word embeddings.
"""

#inserting the vector before given index, along axis 0
embedding_matrix= np.insert(embedding_matrix, [new_index], [new_vector], axis=0)

#updating the indexes of words that follow the new word
word2id= {word: (index+1) if index >= new_index else index
           for word, index in word2id.items()}
word2id[new_word] = new_index
return embedding_matrix, word2id

UNK_INDEX=0 # it is generally common to associate UNK with index 0
UNK_TOKEN="UNK"

embedding_matrix = word_vectors.vectors
unk_vector = embedding_matrix.mean(0)
embedding_matrix, word2id= add_new_word(UNK_TOKEN, unk_vector,
                                         UNK_INDEX, embedding_matrix, word2id)

In [ ]:
def get_int_data(tagged_words, word2id, tag2id):
"""
Replaces all words and tags with their corresponding ids and
seperates words(features) from the tags(labels).
"""

X, Y = [], []
unk_count = 0

for word, tag in tagged_words:
  Y.append(tag2id.get(tag))
  if word in word2id:
      X.append(UNK_INDEX)
      unk_count += 1
   print("Data created. Percentage of unknown words: %.3f" %(unk_count/len(tagged_words)))
   return np.array(X), np.array(Y)

X_train, Y_train = get_int_data(train_words, word2id,tag2id)
X_test, Y_test = get_int_data(test_words, word2id, tag2id)
Y_train, Y_test =to_categorical(Y_train), to_categorical(Y_test)


In [ ]:

def define_model(embedding_matrix, class_count):
"""
   Creates and returns a simple part-of-speech model, which
   takes only one word as input.
   """
 vocab_length = len(embedding_matrix)
 model= Sequential()

# A layer which turns word indexes into vectors
model.add(Embedding(input_dim=vocab_length,
                    output_dim=EMB_DIM,
                    weights=[embedding_matrix],
	             input_length=1))

model.add(Flatten())
model.add(Dense(HIDDEN_SIZE))
model.add(Activation("tanh"))
model.add(Dense(class_count))
model.add(Activation("softmax"))

model.compile(optimizer=tf.train.AdamOptimizer(),
               loss="categorical_crossentropy",
               metrics=["accuracy"])
return model

pos_model = define_model(embedding_matrix, len(tag2id))
pos_model.summary()

#Training the model
pos_model.fit(X_train,
              Y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              verbose=1)

In [ ]:
#Experiment 2-Task 2: List the named entities that you want a model to predict and 
#check if the model predicts those named entities for a sample sentence.
def eva1uate_model(model, id2word, x_test, y_test):
"""
Evaluates the given model by computing the accuracy of its predictions
on the given test data and prints out 10 most mistagged words.
"""

_, acc = model.evaluate(x_test, y_test)
print("Accuracy: %.2f" %acc)

y_pred = model.predict_classes(x_test)
error_counter = collections.Counter()

for i in range(len(x_test)):
   correct_tag_id = np.argmax(y_test[i])
   if y_pred[i]! = correct_tag_id:
      word = id2word[x_test[i]]
      error_counter[word] +=1
   print("Most common errors:\n", error_counter.most_common(10))

id2word = sorted(word2id, key=wordid.get)
evaluate_model(pos_model, id2word, X_test, Y_test)


In [ ]:
EOS_INDEX = 1
EOS_TOKEN = "EOS"

#creating a random end-of-sequence vector
eos_vector = np.random.standard_normal(EMB_DIM)
embedding_matrix, word2id = add_new_word(EOS_TOKEN, eos_vector, EOS_INDEX,embedding_matrix, word2id)



In [ ]:
CONTEXT_SIZE = 2 #define the size of the context-window

def get_window_int_data(tagged_words, word2id, tag2id):

X,Y = [], []
unk_count = 0

span= 2* CONTEXT_SIZE + 1
buffer = collections.deque(maxlen=span)
padding = [(EOS_TOKEN, None)] * CONTEXT_SIZE
buffer +=padding + tagged_words[:CONTEXT_SIZE]

for item in (tagged_words[context_size:] + padding):
  buffer.append(item)

# the input to the model is the ids of all words in the window
window_ids = np.array([word2id.get(word) if (word in word2id) else UNK_INDEX
                       for (word, _) in buffer])
X.append(window_ids)

#the label is the tag of the middle word
middle_word, middle_tag = buffer[CONTEXT_SIZE]
Y.append(tag2id.get(middle_tag))

if middle_word not in word2id:
unk_count +=1
print("Data created. Percentage of unknown words: %.3f" %(unk_count/len(tagged_words)))
return np.array(X), np.array(Y)


In [ ]:
def define_context_sensitive_model(embedding_matrix, class_count):
"""
Creates and returns a part-of-speech model, which
takes as input a tagged word and its context.
"""
vocab_length = len(embedding_matrix)
total_span = CONTEXT_SIZE*2 +1

model=Sequential()
model.add(Embedding(input_dim=vocab_length, output_dim=EMB_DIM, weights=[embedding_matrix],
                    input_length=total_span)),
model.add(Flatten())
model.add(Dense(HIDDEN_SIZE))
model.add(Activation("tanh"))
model.add(Dense(class_count))
model.add(Activation("softmax"))

model.compile(optimizer=tf.train.AdamOptimizer(),
               loss="categorical_crossentropy",
               metrics=["accuracy"])
return model

def evaluate_model(model, id2word, x_test, y_test):
"""
Evaluates the given model by computing the accuracy of its predictions
on the given test data and prints out 10 most mistagged words.
"""

_, acc = model.evaluate(x_test, y_test)  #get accuracy of the model
print("Accuracy: %.2f" % acc)

#the following lines are used to get most commonly mistagged words
y_pred= model.predict_classes(x_test) #get model predictions
error_counter = collections.Counter() #we will use a counter instance


for i in range(len(x_test)):
  correct_tag_id = np.argmax(y_test[i])
  if y_new[i]!= correct_tag_id:
  if isinstance(x_test[i], np.ndarray):
      word = id2word[x_test[i][CONTEXT_SIZE]]
      else:
      word= id2word[x_test[i]]
     error_counter[word] += 1
print("Most common errors:\n" , error_counter.most_common(10))


In [ ]:
X_train2, Y_train2 = get_window_int_data(train_words, word2id, tag2id)
X_test2, Y_test2= get_window_int_data(test_words, word2id, tag2id)
Y_train2, Y_test2 = to_categorical(Y_train2), to_categorical(Y_test2)

cs_pos_model = define_context_sensitive_model(embedding_matrix, len(tag2id))
cs_pos_model.fit(X_train2, Y_train2, batch_size =BATCH_SIZE, epochs=1, verbose=1)

evaluate_model(cs_pos_model, id2word, X_test2, Y_test2) 